## 1) Libraries Installation
##### The cell below is to help you keep track the libraries used and install them quickly.
##### Ensure the correct library names are used, and follow the syntax: **%pip install PACKAGE_NAME**.

In [3]:
install.packages('tsibble')
install.packages('fable')
install.packages('fabletools')
library(tidyverse)
library(tsibble)
library(fable)
library(fabletools)
library(stats)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘quadprog’, ‘progressr’, ‘ggdist’, ‘numDeriv’, ‘fabletools’, ‘distributional’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: fabletools



## 2) Main Section for Code
### **ALL code for machine learning and dataset analysis** should be entered below.
##### Ensure that your code is clear and readable.
##### Remember to include comments and markdown notes as necessary to explain and highlight important segments of your code.

In [10]:
###...code...###
system("git pull")
data = read.csv("Quarterly Data.csv")
data = data[-c(1:2),]
gdp <- data %>%
  select('sasdate', "GDPC1", 'GS10TB3Mx')

gdp$sasdate <- yearquarter(gdp$sasdate)

gdp_ts <- as_tsibble(gdp , index=sasdate)

gdp_ts$logGDP <- log(gdp_ts$GDPC1)
gdp_ts$GDPGR <- 400 * difference(gdp_ts$logGDP, lag=1, differences=1) # Annualised GDP growth rate
gdpgr_plot <- ggplot(gdp_ts, aes(x=sasdate, y=GDPGR)) + geom_line() # Plot of annualised GDP growth rate
gdpgr_plot + labs(x="Year", y = "Annualised GDP growth rate")

ar1 <- gdp_ts[-c(1),] %>%
  model(AR(GDPGR ~ order(2) + lag(GS10TB3Mx)))

report(ar1)

fitted_ar1 <- ar1 %>%
  fitted() %>%
  select("sasdate", ".fitted")

gdp_ts <-gdp_ts %>%
  left_join(fitted_ar1, by=c("sasdate"="sasdate"))

gdp_ts <- gdp_ts %>%
  mutate(mean_sq=(GDPGR - .fitted)^2) %>%
  mutate(recession=ifelse(GDPGR<0, 1, 0))

Warning message in file(file, "rt"):
“cannot open file 'Quarterly Data.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


#### Remember to rename your file name to **NUS_DSESC_DATABUSTERS_XX.ipynb** and ensure that it can run successfully. Good luck and have fun!